# Vanilla

In [4]:
import os
from tsfresh import extract_relevant_features
from tsfresh.examples.robot_execution_failures import (
    download_robot_execution_failures, load_robot_execution_failures
)

os.environ["http_proxy"] = "http://127.0.0.1:10808"
os.environ["https_proxy"] = "http://127.0.0.1:10808"
download_robot_execution_failures()

timeseries, y = load_robot_execution_failures()

In [5]:
timeseries

,id,time,F_x,F_y,F_z,T_x,T_y,T_z
0,1,0,-1,-1,63,-3,-1,0
1,1,1,0,0,62,-3,-1,0
2,1,2,-1,-1,61,-3,0,0
3,1,3,-1,-1,63,-2,-1,0
4,1,4,-1,-1,63,-3,-1,0
...,...,...,...,...,...,...,...,...
1315,88,10,-10,2,39,-21,-24,5
1316,88,11,-11,2,38,-24,-22,6
1317,88,12,-12,3,23,-24,-24,5
1318,88,13,-13,4,26,-29,-27,5


In [6]:
y

1      True
2      True
3      True
4      True
5      True
      ...  
84    False
85    False
86    False
87    False
88    False
Length: 88, dtype: bool

In [7]:
features = extract_relevant_features(timeseries, y, column_id="id", column_sort="time")

Feature Extraction: 100%|██████████| 38/38 [00:02<00:00, 14.13it/s]


In [8]:
features

,F_x__value_count__value_-1,F_x__abs_energy,F_x__root_mean_square,T_y__absolute_maximum,F_x__mean_n_absolute_max__number_of_maxima_7,F_x__range_count__max_1__min_-1,F_y__root_mean_square,F_y__abs_energy,F_y__mean_n_absolute_max__number_of_maxima_7,T_y__standard_deviation,...,"T_x__change_quantiles__f_agg_""var""__isabs_True__qh_0.2__ql_0.0","F_z__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8",T_x__quantile__q_0.1,F_y__has_duplicate_max,T_y__lempel_ziv_complexity__bins_3,T_y__quantile__q_0.1,F_z__time_reversal_asymmetry_statistic__lag_1,F_x__quantile__q_0.2,F_y__quantile__q_0.7,"T_x__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0"
1,14.0,14.0,0.966092,1.0,1.000000,15.0,0.930949,13.0,1.000000,0.471405,...,0.000000,0.0,-3.0,1.0,0.400000,-1.0,-5.960000e+02,-1.0,-1.0,0.000000
2,7.0,25.0,1.290994,5.0,1.571429,13.0,2.250926,76.0,3.000000,2.054805,...,0.000000,1.0,-9.2,1.0,0.533333,-3.6,-6.803846e+02,-1.0,-1.0,0.000000
3,11.0,12.0,0.894427,5.0,1.000000,14.0,1.632993,40.0,2.142857,1.768867,...,0.000000,3.0,-6.6,0.0,0.533333,-4.0,-6.170000e+02,-1.0,0.0,0.000000
4,5.0,16.0,1.032796,6.0,1.285714,10.0,2.000000,60.0,2.428571,2.669998,...,0.000000,0.0,-9.0,0.0,0.533333,-4.6,3.426308e+03,-1.0,1.0,0.000000
5,9.0,17.0,1.064581,5.0,1.285714,13.0,1.751190,46.0,2.285714,2.039608,...,0.000000,0.0,-9.6,0.0,0.466667,-5.0,-2.609000e+03,-1.0,0.8,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,0.0,96833.0,80.346334,167.0,105.285714,0.0,53.404120,42780.0,71.428571,39.541483,...,64.000000,46.0,203.2,0.0,0.533333,36.4,-7.700628e+07,-105.0,66.8,64.000000
85,0.0,1683.0,10.592450,14.0,13.714286,0.0,10.076375,1523.0,12.142857,3.841296,...,4.666667,4.5,-41.6,0.0,0.466667,1.0,-1.050785e+04,5.8,10.6,13.555556
86,0.0,83497.0,74.608757,191.0,98.142857,0.0,37.473546,21064.0,47.714286,52.807154,...,0.250000,7.0,-84.8,0.0,0.466667,19.6,-5.544922e+06,30.4,38.4,0.250000
87,0.0,1405437.0,306.097697,471.0,340.000000,0.0,143.447551,308658.0,157.285714,80.098162,...,0.000000,90.5,-139.2,0.0,0.466667,272.6,-9.881845e+07,246.8,154.8,0.000000


# Using Stock Data

In [17]:
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from tsfresh.utilities.dataframe_functions import make_forecasting_frame, roll_time_series

In [10]:
load_dotenv()  # take environment variables from .env.

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

alchemyEngine = create_engine(
    db_url,
    pool_recycle=3600,
)

In [11]:
# query = """
# with cte as (
# SELECT "date", "open", "close", high, low, volume, amount, open_preclose_rate, high_preclose_rate, low_preclose_rate, vol_change_rate, amt_change_rate, change_rate
# FROM index_daily_em_view
# where symbol = '399673'
# order by date desc
# limit 1200
# ) select * from cte order by date
# """
query = """
with cte as (
SELECT "date", "open", "close", high, low, volume, amount, open_preclose_rate, high_preclose_rate, low_preclose_rate, vol_change_rate, amt_change_rate, change_rate
FROM index_daily_em_view 
where symbol = '399673'
and change_rate is not null
) select * from cte order by date
"""

raw_df = pd.read_sql(query, alchemyEngine, parse_dates=["date"])

In [12]:
df = raw_df.rename(columns={"date": "ds", "change_rate": "y"})
df.insert(0, "unique_id", "399673")

In [15]:
ts_x, ts_y = make_forecasting_frame(df, kind="y", rolling_direction=1, max_timeshift=50)

ValueError: 2